# 编程练习 2：逻辑回归

## 介绍

在本次练习中，你将实现逻辑回归并将其应用于两个不同的数据集。在开始编程练习之前，我们强烈建议你观看相关主题的视频讲座并完成复习问题。

解决此作业所需的所有信息都在此笔记本中，你将实现的所有代码也将在此笔记本中完成。

在开始练习之前，我们需要导入本次编程练习所需的所有库。在整个课程中，我们将使用 [`numpy`](http://www.numpy.org/) 进行所有数组和矩阵操作，使用 [`matplotlib`](https://matplotlib.org/) 进行绘图。在本次作业中，我们还将使用 [`scipy`](https://docs.scipy.org/doc/scipy/reference/)，它包含科学和数值计算的函数和工具。


In [91]:
# 用于操作目录路径
import os
import sys
sys.path.append('.')

# Python 的科学计算和向量计算库
import numpy as np

# 绘图库
from matplotlib import pyplot

# Scipy 中的优化模块
from scipy import optimize

# 为本次练习编写的库，提供用于作业提交和其他功能的附加函数
import utils

# 为本次练习定义提交/评分对象
grader = utils.Grader()

# 告诉 matplotlib 将图嵌入到笔记本中
%matplotlib inline

## 提交与评分

在完成作业的每个部分后，请务必将您的解决方案提交给评分系统。以下是本次练习中每个部分的评分细则。

| 部分   | 内容                                 | 提交函数               | 分数 
| :-     |:-                                    | :-                    | :-:
| 1      | [Sigmoid 函数](#section1)            | [`sigmoid`](#sigmoid) | 10      
| 2      | [计算逻辑回归的代价](#section2)      | [`costFunction`](#costFunction) | 20     
| 3      | [逻辑回归的梯度](#section2)          | [`costFunction`](#costFunction) | 20     
| 4      | [预测函数](#section4)                | [`predict`](#predict) | 10      
| 5      | [计算正则化逻辑回归的代价](#section5) | [`costFunctionReg`](#costFunctionReg) | 20     
| 6      | [正则化逻辑回归的梯度](#section5)    | [`costFunctionReg`](#costFunctionReg) | 20    
|        | 总分                                | | 100    

您可以多次提交您的解决方案，我们将只考虑最高分。

<div class="alert alert-block alert-warning">
在本笔记本的每个部分末尾，我们都有一个包含代码的单元格，用于将当前部分的解决方案提交给评分系统。执行该单元格以查看您到目前为止的得分。为了确保您的所有工作被正确提交，您必须至少执行这些单元格一次。每次更新提交的函数时，也必须重新执行这些单元格。
</div>

1 逻辑回归

在本次练习中，您将构建一个逻辑回归模型来预测学生是否会被大学录取。假设您是一所大学部门的管理员，您希望根据申请人在两次考试中的成绩来确定每位申请人的录取概率。您可以使用来自以前申请者的历史数据作为逻辑回归的训练集。对于每个训练样本，您都有申请人的两次考试成绩和录取决定。您的任务是基于这两次考试的成绩构建一个分类模型，以估计申请人的录取概率。

以下单元格将加载数据及其对应的标签：


In [92]:
# 加载数据
# 前两列包含考试成绩，第三列包含标签。
data = np.loadtxt(os.path.join('Data', 'ex2data1.txt'), delimiter=',')
X, y = data[:, 0:2], data[:, 2]


### 1.1 可视化数据

在开始实现任何学习算法之前，如果可能的话，先可视化数据总是一个好主意。我们通过调用函数 `plotData` 在二维图上显示数据。现在您将完成 `plotData` 中的代码，以便它显示一个图形，其中坐标轴是两次考试的分数，正例和负例用不同的标记显示。

为了帮助您更熟悉绘图，我们将 `plotData` 留空，以便您可以尝试自己实现它。然而，这是一个可选的（不计分的）练习。我们还提供了我们的实现，您可以复制或参考它。如果您选择复制我们的示例，请确保通过查阅 `matplotlib` 和 `numpy` 文档了解其每个命令的作用。

```python
# 找到正例和负例的索引
pos = y == 1
neg = y == 0

# 绘制示例
pyplot.plot(X[pos, 0], X[pos, 1], 'k*', lw=2, ms=10)
pyplot.plot(X[neg, 0], X[neg, 1], 'ko', mfc='y', ms=8, mec='k', mew=1)
```

In [93]:
def plotData(X, y):
    """
    将数据点 X 和 y 绘制到一个新图中。使用 * 表示正例，使用 o 表示负例。
    
    参数
    ----------
    X : array_like
        一个 Mx2 矩阵，表示数据集。
    
    y : array_like
        数据集的标签值。大小为 (M, ) 的向量。
    
    说明
    ------------
    在二维图上绘制正例和负例，正例使用 'k*'，负例使用 'ko'。
    """
    # 创建新图
    fig = pyplot.figure()

    # ====================== 在此处填写您的代码 ======================
    # 找到正例和负例的索引
    pos = y == 1
    neg = y == 0

    # 绘制正例和负例
    pyplot.plot(X[pos, 0], X[pos, 1], 'k*', lw=2, ms=10)
    pyplot.plot(X[neg, 0], X[neg, 1], 'ko', mfc='y', ms=8, mec='k', mew=1)
    # ============================================================


现在，我们调用已实现的函数来显示加载的数据：

In [94]:
plotData(X, y)
# add axes labels
pyplot.xlabel('Exam 1 score')
pyplot.ylabel('Exam 2 score')
pyplot.legend(['Admitted', 'Not admitted'])
pass

<a id="section1"></a>
### 1.2 实现

#### 1.2.1 热身练习：Sigmoid 函数

在开始实际的代价函数之前，回顾一下逻辑回归假设定义为：

$$ h_\theta(x) = g(\theta^T x)$$

其中函数 $g$ 是 Sigmoid 函数。Sigmoid 函数定义为：

$$ g(z) = \frac{1}{1+e^{-z}}$$

您的第一步是实现这个函数 `sigmoid`，以便它可以被程序的其他部分调用。当您完成后，可以通过在新单元格中调用 `sigmoid(x)` 来测试一些值。对于较大的正值 `x`，Sigmoid 应接近 1，而对于较大的负值，Sigmoid 应接近 0。计算 `sigmoid(0)` 应该正好得到 0.5。您的代码还应该支持向量和矩阵的输入。**对于矩阵，您的函数应对每个元素执行 Sigmoid 运算。**
<a id="sigmoid"></a>


In [ ]:
def sigmoid(z):
    """
    计算输入 z 的 Sigmoid 函数。
    
    参数
    ----------
    z : array_like
        Sigmoid 函数的输入。这可以是一个一维向量或二维矩阵。
    
    返回值
    -------
    g : array_like
        计算得到的 Sigmoid 函数值。g 的形状与 z 相同，因为 Sigmoid 是对 z 的每个元素逐一计算的。
        
    说明
    ------------
    计算 z 的每个值的 Sigmoid（z 可以是矩阵、向量或标量）。
    """
    # 将输入转换为 numpy 数组
    z = np.array(z)
    
    # 您需要正确返回以下变量
    g = np.zeros(z.shape)

    # ====================== 在此处填写您的代码 ======================
    # 计算 Sigmoid 函数值
    g = 1 / (1 + np.exp(-z))

    # =============================================================
    return g


以下单元格在 `z=0` 时计算 Sigmoid 函数的值。您应该得到一个值为 0.5。您还可以尝试不同的 `z` 值来实验 Sigmoid 函数。

In [96]:
# 在此处测试 sigmoid 函数的实现
z = 0
g = sigmoid(z)

print('g(', z, ') = ', g)

g( 0 ) =  0.5


在完成练习的某一部分后，您可以通过首先将您修改的函数添加到提交对象中，然后将您的函数发送到 Coursera 进行评分来提交您的解决方案。

提交脚本会提示您输入登录电子邮件和提交令牌。您可以从作业的网页上获取提交令牌。您可以多次提交您的解决方案，我们将只考虑最高分。

执行以下单元格以对练习第一部分的解决方案进行评分。

*您现在应该提交您的解决方案。*

In [97]:
# 将第 1 部分中实现的函数添加到 grader 对象中
grader[1] = sigmoid

# 将添加的函数发送到 coursera 的评分系统以获取此部分的分数
grader.grade()


本地评分结果

                                  Part Name | Score    | Result
                                  --------- | -------- | --------
                           Sigmoid Function | 10       | 通过
                   Logistic Regression Cost | 0        | 未通过
               Logistic Regression Gradient | 0        | 未通过
                                    Predict | 0        | 未通过
       Regularized Logistic Regression Cost | 0        | 未通过
   Regularized Logistic Regression Gradient | 0        | 未通过

总分: 10 / 100


<a id="section2"></a>
#### 1.2.2 成本函数和梯度

现在，您将实现逻辑回归的成本函数和梯度。在继续之前，我们需要为 X 添加截距项。

In [98]:
# 设置数据矩阵，并为截距项添加一列全为 1 的值
m, n = X.shape

# 为 X 添加截距项
X = np.concatenate([np.ones((m, 1)), X], axis=1)

现在，完成函数 `costFunction` 的代码以返回代价和梯度。回顾一下，逻辑回归中的代价函数为：

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \left[ -y^{(i)} \log\left(h_\theta\left( x^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - h_\theta\left( x^{(i)} \right) \right) \right]$$

代价的梯度是一个与 $\theta$ 长度相同的向量，其中第 $j$ 个元素（对于 $j = 0, 1, \cdots , n$）定义如下：

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} $$

注意，虽然这个梯度看起来与线性回归的梯度相同，但公式实际上是不同的，因为线性回归和逻辑回归对 $h_\theta(x)$ 的定义不同。
<a id="costFunction"></a>


In [99]:
def costFunction(theta, X, y):
    """
    计算逻辑回归的代价和梯度。
    
    参数
    ----------
    theta : array_like
        逻辑回归的参数。这是一个形状为 (n+1, ) 的向量。
    
    X : array_like
        输入数据集，形状为 (m x n+1)，其中 m 是数据点的总数，n 是特征的数量。
        我们假设截距项已经添加到输入中。
    
    y : array_like
        输入的标签。这是一个形状为 (m, ) 的向量。
    
    返回值
    -------
    J : float
        代价函数的计算值。
    
    grad : array_like
        一个形状为 (n+1, ) 的向量，表示代价函数相对于 theta 的梯度，
        在当前 theta 值下计算。
        
    说明
    ------------
    计算某个 theta 选择的代价。您应该将 J 设置为代价。
    计算偏导数，并将 grad 设置为代价函数相对于 theta 中每个参数的偏导数。
    """
    # 初始化一些有用的值
    m = y.size  # 训练样本的数量

    # 您需要正确返回以下变量 
    J = 0
    grad = np.zeros(theta.shape)

    # ====================== 在此处填写您的代码 ======================
    
    # 计算 Sigmoid 函数值
    h = sigmoid(X.dot(theta))
    
    # 计算代价函数
    J = -1/m * (y.dot(np.log(h)) + (1-y).dot(np.log(1-h)))
    
    # 计算梯度
    grad = 1/m * X.T.dot(h - y)
    
    # =============================================================
    return J, grad


完成后，通过执行下一个单元格，使用两个测试用例调用您的 `costFunction`。

In [100]:
# 初始化拟合参数
initial_theta = np.zeros(n+1)

cost, grad = costFunction(initial_theta, X, y)

print('Cost at initial theta (zeros): {:.3f}'.format(cost))
print('Expected cost (approx): 0.693\n')

print('Gradient at initial theta (zeros):')
print('\t[{:.4f}, {:.4f}, {:.4f}]'.format(*grad))
print('Expected gradients (approx):\n\t[-0.1000, -12.0092, -11.2628]\n')

# 使用非零 theta 计算并显示代价和梯度
test_theta = np.array([-24, 0.2, 0.2])
cost, grad = costFunction(test_theta, X, y)

print('Cost at test theta: {:.3f}'.format(cost))
print('Expected cost (approx): 0.218\n')

print('Gradient at test theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*grad))
print('Expected gradients (approx):\n\t[0.043, 2.566, 2.647]')

Cost at initial theta (zeros): 0.693
Expected cost (approx): 0.693

Gradient at initial theta (zeros):
	[-0.1000, -12.0092, -11.2628]
Expected gradients (approx):
	[-0.1000, -12.0092, -11.2628]

Cost at test theta: 0.218
Expected cost (approx): 0.218

Gradient at test theta:
	[0.043, 2.566, 2.647]
Expected gradients (approx):
	[0.043, 2.566, 2.647]


*您现在应该提交您的解决方案。*

In [101]:
grader[2] = costFunction
grader[3] = costFunction
grader.grade()


本地评分结果

                                  Part Name | Score    | Result
                                  --------- | -------- | --------
                           Sigmoid Function | 10       | 通过
                   Logistic Regression Cost | 20       | 通过
               Logistic Regression Gradient | 20       | 通过
                                    Predict | 0        | 未通过
       Regularized Logistic Regression Cost | 0        | 未通过
   Regularized Logistic Regression Gradient | 0        | 未通过

总分: 50 / 100


#### 1.2.3 使用 `scipy.optimize` 学习参数

在之前的作业中，您通过实现梯度下降找到了线性回归模型的最优参数。您编写了一个代价函数并计算了其梯度，然后相应地执行了梯度下降步骤。这次，您将不再执行梯度下降步骤，而是使用 [`scipy.optimize` 模块](https://docs.scipy.org/doc/scipy/reference/optimize.html)。SciPy 是一个用于 `python` 的数值计算库。它提供了一个用于求解方程和最小化问题的优化模块。从 `scipy 1.0` 开始，函数 `scipy.optimize.minimize` 是用于解决优化问题（包括约束和无约束问题）的推荐方法。

对于逻辑回归，您需要优化代价函数 $J(\theta)$ 的参数 $\theta$。
具体来说，您将使用 `optimize.minimize` 来找到逻辑回归代价函数的最佳参数 $\theta$，给定一个固定的数据集（X 和 y 值）。您将向 `optimize.minimize` 传递以下输入：
- `costFunction`: 一个代价函数，当给定训练集和特定的 $\theta$ 时，计算数据集 (X, y) 的逻辑回归代价和相对于 $\theta$ 的梯度。需要注意的是，我们只传递函数的名称，而不加括号。这表明我们仅提供对该函数的引用，而不是评估该函数的结果。
- `initial_theta`: 我们试图优化的参数的初始值。
- `(X, y)`: 这些是代价函数的附加参数。
- `jac`: 指示代价函数是否返回雅可比矩阵（梯度）以及代价值。（True）
- `method`: 要使用的优化方法/算法。
- `options`: 可能特定于具体优化方法的附加选项。在下面的代码中，我们仅告诉算法在终止之前的最大迭代次数。

如果您正确完成了 `costFunction`，`optimize.minimize` 将会收敛到正确的优化参数，并在一个类对象中返回代价和 $\theta$ 的最终值。请注意，通过使用 `optimize.minimize`，您无需自己编写任何循环，也无需像梯度下降那样设置学习率。这些都由 `optimize.minimize` 完成：您只需提供一个计算代价和梯度的函数即可。

在下面的代码中，我们已经编写了调用 `optimize.minimize` 的代码，并传递了正确的参数。

In [102]:
# 设置 optimize.minimize 的选项
options= {'maxiter': 400}

# 查看 scipy 的 optimize.minimize 文档，了解不同参数的描述
# 该函数返回一个 `OptimizeResult` 对象
# 我们使用截断牛顿算法进行优化，这相当于 MATLAB 的 fminunc
# 参见 https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy
res = optimize.minimize(costFunction,
                        initial_theta,
                        (X, y),
                        jac=True,
                        method='TNC',
                        options=options)

# `OptimizeResult` 对象的 fun 属性返回
# 在优化后的 theta 下 costFunction 的值
cost = res.fun

# 优化后的 theta 存储在 x 属性中
theta = res.x

# 打印优化后的 theta 到屏幕
print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))
print('Expected cost (approx): 0.203\n');

print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*theta))
print('Expected theta (approx):\n\t[-25.161, 0.206, 0.201]')


Cost at theta found by optimize.minimize: 0.203
Expected cost (approx): 0.203

theta:
	[-25.161, 0.206, 0.201]
Expected theta (approx):
	[-25.161, 0.206, 0.201]


C:\Users\33530\AppData\Local\Temp\ipykernel_23316\1469082979.py:8: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,


一旦 `optimize.minimize` 完成，我们希望使用最终的 $\theta$ 值在训练数据上可视化决策边界，如下图所示。

![](Figures/decision_boundary1.png)

为此，我们编写了一个函数 `plotDecisionBoundary`，用于在训练数据上绘制决策边界。您不需要编写任何代码来绘制决策边界，但我们也鼓励您查看 `plotDecisionBoundary` 中的代码，以了解如何使用 $\theta$ 值绘制这样的边界。您可以在随本次作业提供的 `utils.py` 文件中找到此函数。

In [103]:
# 绘制边界
utils.plotDecisionBoundary(plotData, theta, X, y)

<a id="section4"></a>
#### 1.2.4 评估逻辑回归

在学习参数后，您可以使用模型预测某个学生是否会被录取。对于一名考试 1 分数为 45 和考试 2 分数为 85 的学生，您应该期望看到录取概率为 0.776。评估我们找到的参数质量的另一种方法是查看学习到的模型在我们的训练集上的预测效果。在这一部分，您的任务是完成函数 `predict` 中的代码。`predict` 函数将根据数据集和学习到的参数向量 $\theta$ 生成“1”或“0”的预测。
<a id="predict"></a>


In [104]:
def predict(theta, X):
    """
    使用学习到的逻辑回归预测标签是 0 还是 1。
    使用阈值 0.5 计算 X 的预测值 
    （即，如果 sigmoid(theta.T*x) >= 0.5，则预测为 1）
    
    参数
    ----------
    theta : array_like
        逻辑回归的参数。形状为 (n+1, ) 的向量。
    
    X : array_like
        用于计算预测的数据。行数是要计算预测的点的数量，
        列数是特征的数量。

    返回值
    -------
    p : array_like
        每行 X 的预测值，0 或 1。
    
    说明
    ------------
    完成以下代码以使用学习到的逻辑回归参数进行预测。
    您应该将 p 设置为一个由 0 和 1 组成的向量。
    """
    m = X.shape[0] # 训练样本的数量

    # 您需要正确返回以下变量
    p = np.zeros(m)

    # ====================== 在此处填写您的代码 ======================
    # 计算 Sigmoid 函数值
    h = sigmoid(X.dot(theta))
    
    # 计算预测值
    p = h >= 0.5
    
    # ============================================================
    return p


在您完成 `predict` 中的代码后，我们继续通过计算分类器正确分类的样本百分比来报告训练准确率。

In [105]:
# 预测考试 1 分数为 45 和考试 2 分数为 85 的学生的录取概率
prob = sigmoid(np.dot([1, 45, 85], theta))
print('For a student with scores 45 and 85,'
      'we predict an admission probability of {:.3f}'.format(prob))
print('Expected value: 0.775 +/- 0.002\n')

# 计算我们的训练集上的准确率
p = predict(theta, X)
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y) * 100))
print('Expected accuracy (approx): 89.00 %')


For a student with scores 45 and 85,we predict an admission probability of 0.776
Expected value: 0.775 +/- 0.002

Train Accuracy: 89.00 %
Expected accuracy (approx): 89.00 %


*您现在应该提交您的解决方案。*

In [106]:
grader[4] = predict
grader.grade()


本地评分结果

                                  Part Name | Score    | Result
                                  --------- | -------- | --------
                           Sigmoid Function | 10       | 通过
                   Logistic Regression Cost | 20       | 通过
               Logistic Regression Gradient | 20       | 通过
                                    Predict | 0        | 未通过
       Regularized Logistic Regression Cost | 0        | 未通过
   Regularized Logistic Regression Gradient | 0        | 未通过

总分: 50 / 100


## 2 正则化逻辑回归

在本部分练习中，您将实现正则化逻辑回归，以预测来自制造工厂的微芯片是否通过质量保证（QA）。在 QA 过程中，每个微芯片都会经过各种测试，以确保其正常运行。
假设您是工厂的产品经理，并且您拥有一些微芯片在两个不同测试中的测试结果。通过这两个测试，您希望确定是否应该接受或拒绝这些微芯片。为了帮助您做出决定，您拥有一个关于过去微芯片测试结果的数据集，您可以基于此构建一个逻辑回归模型。

首先，我们从 CSV 文件中加载数据：

In [107]:
# 加载数据
# 前两列包含 X 值，第三列包含标签 (y)。
data = np.loadtxt(os.path.join('Data', 'ex2data2.txt'), delimiter=',')
X = data[:, :2]
y = data[:, 2]

### 2.1 可视化数据

与本次练习的前面部分类似，`plotData` 用于生成一个图形，其中坐标轴是两个测试分数，正例（y = 1，接受）和负例（y = 0，拒绝）用不同的标记显示。


In [108]:
plotData(X, y)
# 标签和图例
pyplot.xlabel('Microchip Test 1')  # 微芯片测试 1
pyplot.ylabel('Microchip Test 2')  # 微芯片测试 2

# 按绘图顺序指定
pyplot.legend(['y = 1', 'y = 0'], loc='upper right')  # 图例位置：右上角
pass

上图显示，我们的数据集无法通过图中的直线将正例和负例分开。因此，直接应用逻辑回归在此数据集上表现不会很好，因为逻辑回归只能找到线性决策边界。

### 2.2 特征映射

一种更好地拟合数据的方法是从每个数据点创建更多的特征。在文件 `utils.py` 中定义的函数 `mapFeature` 中，我们将特征映射到 $x_1$ 和 $x_2$ 的所有多项式项，直到六次幂。

$$ \text{mapFeature}(x) = \begin{bmatrix} 1 & x_1 & x_2 & x_1^2 & x_1 x_2 & x_2^2 & x_1^3 & \dots & x_1 x_2^5 & x_2^6 \end{bmatrix}^T $$

通过这种映射，我们的两个特征向量（两个 QA 测试的分数）被转换为一个 28 维的向量。在这个高维特征向量上训练的逻辑回归分类器将具有更复杂的决策边界，并且在我们的二维图中绘制时将呈现非线性。
虽然特征映射允许我们构建更具表现力的分类器，但它也更容易过拟合。在练习的下一部分中，您将实现正则化逻辑回归以拟合数据，并亲自观察正则化如何帮助解决过拟合问题。


In [109]:
# 请注意，mapFeature 还为我们添加了一列全为 1 的值，因此截距项已被处理
X = utils.mapFeature(X[:, 0], X[:, 1])

<a id="section5"></a>
### 2.3 成本函数和梯度

现在，您将实现代码以计算正则化逻辑回归的成本函数和梯度。完成下面函数 `costFunctionReg` 的代码以返回成本和梯度。

回顾一下，逻辑回归中的正则化成本函数为：

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)}\log \left( h_\theta \left(x^{(i)} \right) \right) - \left( 1 - y^{(i)} \right) \log \left( 1 - h_\theta \left( x^{(i)} \right) \right) \right] + \frac{\lambda}{2m} \sum_{j=1}^n \theta_j^2 $$

注意，您不应该对参数 $\theta_0$ 进行正则化。成本函数的梯度是一个向量，其中第 $j^{th}$ 个元素定义如下：

$$ \frac{\partial J(\theta)}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left(x^{(i)}\right) - y^{(i)} \right) x_j^{(i)} \qquad \text{对于 } j =0 $$

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left(x^{(i)}\right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m}\theta_j \qquad \text{对于 } j \ge 1 $$
<a id="costFunctionReg"></a>

In [110]:
def costFunctionReg(theta, X, y, lambda_):
    """
    计算带正则化的逻辑回归的代价和梯度。
    
    参数
    ----------
    theta : array_like
        逻辑回归的参数。一个形状为 (n, ) 的向量。n 是特征的数量，
        包括任何截距项。如果我们已将初始特征映射到多项式特征，
        那么 n 是多项式特征的总数。
    
    X : array_like
        数据集，形状为 (m x n)。m 是样本数量，n 是特征数量
        （经过特征映射后）。
    
    y : array_like
        数据标签。一个形状为 (m, ) 的向量。
    
    lambda_ : float
        正则化参数。
    
    返回值
    -------
    J : float
        正则化代价函数的计算值。
    
    grad : array_like
        一个形状为 (n, ) 的向量，表示代价函数相对于 theta 的梯度，
        在当前 theta 值下计算。
    
    说明
    ------------
    计算特定 theta 选择的代价 `J`。
    计算偏导数，并将 `grad` 设置为代价函数相对于 theta 中每个参数的偏导数。
    """
    # 初始化一些有用的值
    m = y.size  # 训练样本的数量

    # 您需要正确返回以下变量 
    J = 0
    grad = np.zeros(theta.shape)

    # ===================== 在此处填写您的代码 ======================
    # 计算 Sigmoid 函数值
    h = sigmoid(X.dot(theta))
    
    # 计算代价函数
    J = -1/m * (y.dot(np.log(h)) + (1-y).dot(np.log(1-h))) + lambda_/2/m * theta[1:].dot(theta[1:])
    
    # 计算梯度
    grad = 1/m * X.T.dot(h - y)
    grad[1:] = grad[1:] + lambda_/m * theta[1:]
    
    
    # =============================================================
    return J, grad


完成 `costFunctionReg` 后，我们在下面调用它，使用 $\theta$ 的初始值（初始化为全零），以及另一个测试用例，其中 $\theta$ 全为 1。

In [111]:
# 初始化拟合参数
initial_theta = np.zeros(X.shape[1])

# 设置正则化参数 lambda 为 1
# 不要使用 `lambda` 作为变量名
# 因为它是 Python 的关键字
lambda_ = 1

# 计算并显示正则化逻辑回归的初始代价和梯度
cost, grad = costFunctionReg(initial_theta, X, y, lambda_)

print('Cost at initial theta (zeros): {:.3f}'.format(cost))
print('Expected cost (approx)       : 0.693\n')

print('Gradient at initial theta (zeros) - first five values only:')
print('\t[{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}]'.format(*grad[:5]))
print('Expected gradients (approx) - first five values only:')
print('\t[0.0085, 0.0188, 0.0001, 0.0503, 0.0115]\n')


# 计算并显示代价和梯度
# 使用全为 1 的 theta 和 lambda = 10
test_theta = np.ones(X.shape[1])
cost, grad = costFunctionReg(test_theta, X, y, 10)

print('------------------------------\n')
print('Cost at test theta    : {:.2f}'.format(cost))
print('Expected cost (approx): 3.16\n')

print('Gradient at initial theta (zeros) - first五个值:')
print('\t[{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}]'.format(*grad[:5]))
print('Expected gradients (approx) - first五个值:')
print('\t[0.3460, 0.1614, 0.1948, 0.2269, 0.0922]')


Cost at initial theta (zeros): 0.693
Expected cost (approx)       : 0.693

Gradient at initial theta (zeros) - first five values only:
	[0.0085, 0.0188, 0.0001, 0.0503, 0.0115]
Expected gradients (approx) - first five values only:
	[0.0085, 0.0188, 0.0001, 0.0503, 0.0115]

------------------------------

Cost at test theta    : 3.16
Expected cost (approx): 3.16

Gradient at initial theta (zeros) - first五个值:
	[0.3460, 0.1614, 0.1948, 0.2269, 0.0922]
Expected gradients (approx) - first五个值:
	[0.3460, 0.1614, 0.1948, 0.2269, 0.0922]


*您现在应该提交您的解决方案。*

In [112]:
grader[5] = costFunctionReg
grader[6] = costFunctionReg
grader.grade()


本地评分结果

                                  Part Name | Score    | Result
                                  --------- | -------- | --------
                           Sigmoid Function | 10       | 通过
                   Logistic Regression Cost | 20       | 通过
               Logistic Regression Gradient | 20       | 通过
                                    Predict | 0        | 未通过
       Regularized Logistic Regression Cost | 20       | 通过
   Regularized Logistic Regression Gradient | 20       | 通过

总分: 90 / 100


#### 2.3.1 使用 `scipy.optimize.minimize` 学习参数

与前面的部分类似，您将使用 `optimize.minimize` 来学习最优参数 $\theta$。如果您已经正确完成了正则化逻辑回归的代价和梯度计算（`costFunctionReg`），您应该能够完成接下来的部分，通过 `optimize.minimize` 学习参数 $\theta$。

### 2.4 绘制决策边界

为了帮助您可视化此分类器学习到的模型，我们提供了函数 `plotDecisionBoundary`，该函数绘制了将正例和负例分开的（非线性）决策边界。在 `plotDecisionBoundary` 中，我们通过在均匀分布的网格上计算分类器的预测值，然后绘制从 y = 0 到 y = 1 的预测值变化的等高线图来绘制非线性决策边界。

### 2.5 可选（不计分）练习

在本部分练习中，您将尝试为数据集设置不同的正则化参数，以了解正则化如何防止过拟合。

注意，当您改变 $\lambda$ 时，决策边界的变化。对于较小的 $\lambda$，您会发现分类器几乎可以正确分类每个训练样本，但会绘制一个非常复杂的边界，从而导致过拟合数据。请参阅以下图像，了解不同 $\lambda$ 值下的决策边界。

<table>
    <tr>
        <td style="text-align:center">
            无正则化（过拟合）<img src="Figures/decision_boundary3.png">
        </td>        
        <td style="text-align:center">
            带正则化的决策边界
            <img src="Figures/decision_boundary2.png">
        </td>
        <td style="text-align:center">
            过度正则化的决策边界
            <img src="Figures/decision_boundary4.png">
        </td>        
    <tr>
</table>

这不是一个好的决策边界：例如，它预测在 $x = (−0.25, 1.5)$ 处的点被接受 $(y = 1)$，这似乎是一个错误的决策，考虑到训练集的情况。
对于较大的 $\lambda$，您应该看到一个显示更简单决策边界的图，该边界仍然可以很好地分隔正例和负例。然而，如果 $\lambda$ 设置得过高，您将无法获得良好的拟合，决策边界将无法很好地跟随数据，从而导致欠拟合数据。


In [113]:
# 初始化拟合参数
initial_theta = np.zeros(X.shape[1])

# 将正则化参数 lambda 设置为 1（您可以尝试不同的值）
lambda_ = 1.00

# 为 optimize.minimize 设置选项
options= {'maxiter': 100}

res = optimize.minimize(costFunctionReg,
                        initial_theta,
                        (X, y, lambda_),
                        jac=True,
                        method='TNC',
                        options=options)

# OptimizeResult 对象的 fun 属性返回
# 在优化后的 theta 下 costFunction 的值
cost = res.fun

# 优化后的 theta 存储在结果的 x 属性中
theta = res.x

utils.plotDecisionBoundary(plotData, theta, X, y)
pyplot.xlabel('Microchip Test 1')
pyplot.ylabel('Microchip Test 2')
pyplot.legend(['y = 1', 'y = 0'])
pyplot.grid(False)
pyplot.title('lambda = %0.2f' % lambda_)

# 计算训练集上的准确率
p = predict(theta, X)

print('Train Accuracy: %.1f %%' % (np.mean(p == y) * 100))
print('Expected accuracy (with lambda = 1): 83.1 % (approx)\n')


C:\Users\33530\AppData\Local\Temp\ipykernel_23316\1405272214.py:10: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunctionReg,


Train Accuracy: 83.1 %
Expected accuracy (with lambda = 1): 83.1 % (approx)



*您不需要为这些可选（不计分）的练习提交任何解决方案。*